This notebook is my attempt to copy several Sudoku solving guides online. As well as to experiment with Sudoku Puzzle Theory and Deep Learning

https://www.activestate.com/blog/solving-sudoku-with-python-and-artificial-intelligence/

https://www.kaggle.com/code/yashchoudhary/deep-sudoku-solver-multiple-approaches/notebook

https://www.analyticsvidhya.com/blog/2021/05/solving-sudoku-from-image-using-deep-learning-with-python-code/ 

handle imports

In [4]:
import numpy as np
import pandas as pd
import time
import itertools
#download dataset from https://www.kaggle.com/datasets/rohanrao/sudoku
#gitignore should ignore the large csv file

#sudoku_df = pd.DataFrame(pd.read_csv('sudoku.csv'))
# nrows to limit import size, the file has 9million rows so.... (it took me 20secs)
sudoku_df = pd.DataFrame(pd.read_csv('sudoku.csv', nrows=100))

visualize sudoku puzzle

In [8]:
# this def shapes the string of 81 char into a 9x9 arr
def sudoku_shape(sudoku_df):
    for n in range(sudoku_df.shape[0]):
        sudoku_df.iloc[n,0] = np.reshape(list(sudoku_df.puzzle.values[n]),(9,9)).astype(int)
        sudoku_df.iloc[n,1] = np.reshape(list(sudoku_df.solution.values[n]),(9,9)).astype(int)
    return sudoku_df

#depending on how many rows you imported, this could take a while.
#in the future i might do this for the entire file and save it as some binary or something
sudoku_df = sudoku_shape(sudoku_df)

example 0, unsolved

In [10]:
sudoku_df.iloc[0,0]

array([[0, 7, 0, 0, 0, 0, 0, 4, 3],
       [0, 4, 0, 0, 0, 9, 6, 1, 0],
       [8, 0, 0, 6, 3, 4, 9, 0, 0],
       [0, 9, 4, 0, 5, 2, 0, 0, 0],
       [3, 5, 8, 4, 6, 0, 0, 2, 0],
       [0, 0, 0, 8, 0, 0, 5, 3, 0],
       [0, 8, 0, 0, 7, 0, 0, 9, 1],
       [9, 0, 2, 1, 0, 0, 0, 0, 5],
       [0, 0, 7, 0, 4, 0, 8, 0, 2]])

example 0, solved

In [11]:
sudoku_df.iloc[0,1]

array([[6, 7, 9, 5, 1, 8, 2, 4, 3],
       [5, 4, 3, 7, 2, 9, 6, 1, 8],
       [8, 2, 1, 6, 3, 4, 9, 5, 7],
       [7, 9, 4, 3, 5, 2, 1, 8, 6],
       [3, 5, 8, 4, 6, 1, 7, 2, 9],
       [2, 1, 6, 8, 9, 7, 5, 3, 4],
       [4, 8, 5, 2, 7, 6, 3, 9, 1],
       [9, 6, 2, 1, 8, 3, 4, 7, 5],
       [1, 3, 7, 9, 4, 5, 8, 6, 2]])

check if proposed solution is valid

In [12]:
def check_puzzle(sudoku_puzzle):
    # check for 9 unique digits in row and col
    checkRow = all([all([x in sudoku_puzzle[nrow,:] for x in range(1,10)]) for nrow in range(9)])
    checkCol = all([all([x in sudoku_puzzle[:,ncol] for x in range(1,10)]) for ncol in range(9)])
    
    # check for 9 unique digits in each home
    checkUpperLeft = all([x in sudoku_puzzle[0:3,0:3] for x in range(1,10)])
    checkUpperMid = all([x in sudoku_puzzle[0:3,3:6] for x in range(1,10)])
    checkUpperRight = all([x in sudoku_puzzle[0:3,6:9] for x in range(1,10)])

    checkMidLeft = all([x in sudoku_puzzle[3:6,0:3] for x in range(1,10)])
    checkMidMid = all([x in sudoku_puzzle[3:6,3:6] for x in range(1,10)])
    checkMidRight = all([x in sudoku_puzzle[3:6,6:9] for x in range(1,10)])

    checkLowerLeft = all([x in sudoku_puzzle[6:9,0:3] for x in range(1,10)])
    checkLowerMid = all([x in sudoku_puzzle[6:9,3:6] for x in range(1,10)])
    checkLowerRight = all([x in sudoku_puzzle[6:9,6:9] for x in range(1,10)])

    solved = all([checkRow,checkCol,checkUpperLeft,checkUpperMid,checkUpperRight,
                  checkMidLeft,checkMidMid,checkMidRight,checkLowerLeft,checkLowerMid,checkLowerRight])
    if solved:
        for line in sudoku_puzzle:
            print(*line)
    return solved

In [13]:
check_puzzle(sudoku_df.iloc[0,1])

6 7 9 5 1 8 2 4 3
5 4 3 7 2 9 6 1 8
8 2 1 6 3 4 9 5 7
7 9 4 3 5 2 1 8 6
3 5 8 4 6 1 7 2 9
2 1 6 8 9 7 5 3 4
4 8 5 2 7 6 3 9 1
9 6 2 1 8 3 4 7 5
1 3 7 9 4 5 8 6 2


True